In [1]:
import numpy as np
import os
import pysynphot as s
import pandas as pd

In [2]:
jpas_filters_path = '/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Filters/JPAS_filters'
test_specs        = '/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Sanity_Check/Specs'
results_path      = '/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Sanity_Check/Results'
jpas_filters_list = np.loadtxt('/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Filters/jpas_filters_list.txt', dtype=str)
specs_list        = np.loadtxt('/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Sanity_Check/specslist.txt', dtype=str)

In [3]:
# Constants --------------------------------------------------------------------------------------------------------
c = 2.99792458E18                  # Light Speed in Angstrom s^-1

In [4]:
# Setting the T80 M1 effective area in cm^2 ------------------------------------------------------------------------
s.refs.PRIMARY_AREA = 4400

In [5]:
for each_spectrum in specs_list:
    # Transforming the specs from f_lambda to f_nu -----------------------------------------------------------------
    wavelength = np.loadtxt(os.path.join(test_specs, each_spectrum), usecols=[0])
    f_lambda   = np.loadtxt(os.path.join(test_specs, each_spectrum), usecols=[1])
    # Simulating the photometry for each J-PAS band ----------------------------------------------------------------
    filter_name = []
    photometry  = []
    for jpas_filters in jpas_filters_list:
        filter_name_i          = jpas_filters.split('.')[0]
        filter_name.append(filter_name_i)
        jpas_filter_bandpass = s.FileBandpass(os.path.join(jpas_filters_path, jpas_filters))        
#         print jpas_filter_bandpass.throughput
        sdss_spectrum        = s.FileSpectrum(os.path.join(test_specs, each_spectrum))
        photometry_i         = s.Observation(sdss_spectrum, jpas_filter_bandpass, binset=np.arange(3000, 11000), 
                                             force='extrap')
        photometry.append(photometry_i.countrate())
    filter_name       = np.array(filter_name)
    photometry        = np.array(photometry)
    galaxy_simulation = np.vstack((filter_name, photometry))
    galaxy_simulation = pd.DataFrame(galaxy_simulation)
    galaxy_simulation.to_csv(os.path.join(results_path, os.path.split(each_spectrum)[-1][0:14]+'.csv'), sep=',', 
                             header=None, index=False)

[ 0.00065593  0.00139488  0.00295979  0.00626264  0.0081343   0.01054404
  0.01405466  0.01870098  0.02483877  0.03294183  0.04363748  0.05775705
  0.07224734  0.09035164  0.10514928  0.12237961  0.13910651  0.15810334
  0.17377228  0.19095153  0.20678687  0.22384276  0.23423214  0.24500258
  0.25375593  0.26273665  0.26918924  0.27568025  0.28098228  0.28638887
  0.29191803  0.29758403  0.30191058  0.30634267  0.31088095  0.31552305
  0.31923894  0.32297255  0.32676525  0.33060806  0.33449132  0.33840434
  0.34232186  0.34624594  0.35016525  0.35406971  0.35638586  0.35863126
  0.36082827  0.36298303  0.36510639  0.36721351  0.36931727  0.37144177
  0.37419257  0.37739522  0.38024198  0.38314534  0.38611132  0.38912739
  0.39217426  0.39522754  0.39821228  0.40114982  0.40401744  0.40679951
  0.40901945  0.41112348  0.41315266  0.41513416  0.41710376  0.41910219
  0.42113739  0.4232823   0.42556871  0.42801781  0.43039233  0.43290106
  0.43554618  0.43829024  0.44108412  0.44387091  0